In [1]:
import os
import pandas as pd
import numpy as np
import datetime
from datetime import datetime
import time
import json
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import awswrangler as wr

In [3]:
import preprocessing
import utils

In [4]:
prefix_path = 's3://think-tank-casestudy/preprocessed_data'

In [6]:
train_data = wr.s3.read_parquet(path=f'{prefix_path}/train_data_clustered.parquet')
test_data = wr.s3.read_parquet(path=f'{prefix_path}/test_data_clustered.parquet')

- CALL TYPE -> ONE_HOT ENCODING, no ordinal relationship
- WEATHER --> ONE HOT ENCODING, no ordinal relationship
- ORIGIN STAND --> Reduction of High cardinality + ONE HOT ENCODING
- MONTH/WEEK per year --> ONE HOT ENCODING or ORDINAL ENCODING

In [8]:
train_data.head()

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,WEATHER,TIMESTAMP_DT,N_COORDINATE_POINTS,TOTAL_FLIGHT_TIME_MINUTES,START_POINT,DEST_POINT,TOTAL_DISTANCE_KM,SEQUENCE,START_POINT_LON,START_POINT_LAT,DEST_POINT_LON,DEST_POINT_LAT,cluster_label
0,1372636858620000589,C,NaN,NaN,20000589,Rainy,2013-07-01 02:00:58,23,5.5,"[-8.618643, 41.141412]","[-8.630838, 41.154489]",1.976270,"[-8.618643, 41.141412, -8.618499, 41.141376, -...",-8.618643,41.141412,-8.630838,41.154489,208
1,1372637303620000596,B,NaN,7.0,20000596,Foggy,2013-07-01 02:08:23,19,4.5,"[-8.639847, 41.159826]","[-8.66574, 41.170671]",3.116235,"[-8.639847, 41.159826, -8.640351, 41.159871, -...",-8.639847,41.159826,-8.665740,41.170671,359
2,1372636951620000320,C,NaN,NaN,20000320,Rainy,2013-07-01 02:02:31,65,16.0,"[-8.612964, 41.140359]","[-8.61597, 41.14053]",0.334780,"[-8.612964, 41.140359, -8.613378, 41.14035, -8...",-8.612964,41.140359,-8.615970,41.140530,304
3,1372636854620000520,C,NaN,NaN,20000520,Cloudy,2013-07-01 02:00:54,43,10.5,"[-8.574678, 41.151951]","[-8.607996, 41.142915]",3.835687,"[-8.574678, 41.151951, -8.574705, 41.151942, -...",-8.574678,41.151951,-8.607996,41.142915,343
4,1372637091620000337,C,NaN,NaN,20000337,Windy,2013-07-01 02:04:51,29,7.0,"[-8.645994, 41.18049]","[-8.687268, 41.178087]",4.597055,"[-8.645994, 41.18049, -8.645949, 41.180517, -8...",-8.645994,41.180490,-8.687268,41.178087,653


In [10]:
train_data = utils.reduce_high_cardinality(train_data, ['ORIGIN_STAND'])
test_data = utils.reduce_high_cardinality(test_data, ['ORIGIN_STAND'])

In [14]:
categories_oh = ['CALL_TYPE','WEATHER','ORIGIN_STAND_agg']
#categories_oe = ['YEAR_MONTH']

In [15]:
df_fenc_oh = utils.feature_encoding_oh(train_data, categories_oh)
#df_fenc_oe = utils.feature_encoding_oe(train_data, categories_oe)
train_data = pd.concat([train_data, df_fenc_oh, 
                       # df_fenc_oe
                       ],axis=1)

In [17]:
df_fenc_oh = utils.feature_encoding_oh(test_data, categories_oh)
#df_fenc_oe = feature_encoding_oe(test_data, categories_oe)
test_data = pd.concat([test_data, df_fenc_oh,
                       #df_fenc_oe
                      ],axis=1)

In [19]:
test_data, train_data = utils.add_binary_features(train_data, test_data)

In [20]:
assert(train_data.shape[1] == test_data.shape[1])

In [23]:
non_features = ['CALL_TYPE','ORIGIN_CALL','ORIGIN_STAND', 'START_POINT','DEST_POINT',
                'TIMESTAMP_MONTH','TIMESTAMP_DAY','TIMESTAMP_WEEK','TIMESTAMP_YEAR','YEAR_MONTH','ORIGIN_STAND_agg',
               'MISSING_DATA','POLYLINE','WEATHER','TAXI_ID','TIMESTAMP_DT','TIMESTAMP']

In [26]:
n_cluster = 1000

In [31]:
train_data = train_data[[column_ for column_ in train_data.columns if column_ not in non_features]]
wr.s3.to_parquet(df=train_data, path=f's3://think-tank-casestudy/features_engineered/n_cluster_1000/feature_engineered_train.parquet')
test_data = test_data[[column_ for column_ in train_data.columns if column_ not in non_features]]
wr.s3.to_parquet(df=test_data, path=f's3://think-tank-casestudy/features_engineered/n_cluster_100/feature_engineered_test.parquet')

/Users/julia.hehl/opt/anaconda3/lib/python3.9/site-packages/pandas/core/common.py:245: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = np.asarray(values, dtype=dtype)


TypeError: Index must either be string or integer